## Assign topics to each tweet using the best iteration of each model

In [2]:
import tweetclassifier as tc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import datetime
from time import time

%matplotlib inline
warnings.filterwarnings('ignore')

pd.set_option('display.max_info_columns',200)
pd.set_option('display.max_colwidth',200)
pd.set_option('display.max_rows', 100)
plt.style.use('ggplot')

In [3]:
clf = tc.classifytweet()

Model items loaded and classifier initialized!


In [21]:
df = pd.read_csv('../../data_cleaned_w_sentiment_outrage_counts.csv', sep=',', quotechar='"', index_col=False, header=0)
df.drop(['text_token','text_stemmed','text_processed'], inplace=True, axis=1)
df.head(1)

,user.description,text,topic,log_followers,log_retweets,engagement_flag,positive_emo,outrage_emo,net_emo_outrage,predicted_sentiment,negative_sentiment_prob,num_outrage_words_ext,num_outrage_words
0,"Part of the #VastRightWingConspiracy -- thanks, @HillaryClinton. Better candidate: https://www.randpaul.com/ Retweeting not endorsement. Image: TV lover.","#BanTheBox means ""all is #forgiven""/NO MORE federal #NCIC checks when buying guns? Please consider it, Mr. President @POTUS. #forgiving",G,5.252273,0.0,0,0,0,0,4,0.265578,0,0


### Get the outrage index parameters and scale the scores to [0,1]

In [22]:
#df['base_outrage_count'] = 0
#df['enhanced_outrage_count'] = 0
df['arousal'] = 0
df['valence'] = 0
df['sentiment'] = 0
#df['emoji_count'] = 0
df['topics'] = np.empty((len(df), 0)).tolist()
df['topic_arousal'] = 0
df['topic_valence'] = 0
df['outrage'] = 0

In [5]:
df.head(1)

,user.description,text,topic,log_followers,log_retweets,engagement_flag,positive_emo,outrage_emo,net_emo_outrage,predicted_sentiment,negative_sentiment_prob,num_outrage_words_ext,num_outrage_words,arousal,valence,sentiment,topics,topic_arousal,topic_valence,outrage
0,"Part of the #VastRightWingConspiracy -- thanks, @HillaryClinton. Better candidate: https://www.randpaul.com/ Retweeting not endorsement. Image: TV lover.","#BanTheBox means ""all is #forgiven""/NO MORE federal #NCIC checks when buying guns? Please consider it, Mr. President @POTUS. #forgiving",G,5.252273,0.0,0,0,0,0,4,0.265578,0,0,0,0,0,[],0,0,0


In [13]:
df_sampled = df.sample(frac=.1)
col_map = {v:ix for ix,v in enumerate(df_sampled.columns.tolist())}
tweets = df_sampled.text.tolist()
to_insert = list()
t0 = time()

for ix in range(df_sampled.shape[0]):
    if ix % 2500 == 0:
        elapsed = time() - t0
        print("Processing tweet number %s, time elapsed: %s" %(str(ix+1), str(datetime.timedelta(seconds=elapsed))))
    clf.prepare_tweet(tweets[ix])
    '''
    #df.iloc[ix, col_map['base_outrage_count']] = clf.get_base_outrage_count()
    #df.iloc[ix, col_map['enhanced_outrage_count']] = clf.get_expanded_outrage_count()
    df.iloc[ix, col_map['valence']] = clf.get_valence_score()
    df.iloc[ix, col_map['arousal']] = clf.get_arousal_score()
    df.iloc[ix, col_map['sentiment']] = clf.get_sentiment_score()
    #df.iloc[ix, col_map['emoji_count']] = clf.get_emoji_count()
    
    topic_arousal_score = 0
    topic_valence_score = 0
    for tup in clf.get_topics():
        df.iloc[ix, col_map['topics']].append(tup)
        topic_arousal_score += clf.topic_arousal_scaler[tup[0]] * tup[1]
        topic_valence_score += clf.topic_valence_scaler[tup[0]] * tup[1]
    df.iloc[ix, col_map['topic_arousal']] = topic_arousal_score
    df.iloc[ix, col_map['topic_valence']] = topic_valence_score

    #df.iloc[ix, col_map['outrage']] = clf.get_outrage_score()
    '''    
    topic_arousal_score = 0
    topic_valence_score = 0
    for tup in clf.get_topics():
        df_sampled.iloc[ix, col_map['topics']].append(tup)
        topic_arousal_score += clf.topic_arousal_scaler[tup[0]] * tup[1]
        topic_valence_score += clf.topic_valence_scaler[tup[0]] * tup[1]
    
    list_insert = [
        clf.get_arousal_score(),
        clf.get_valence_score(),
        clf.get_sentiment_score(),
        clf.get_topics(),
        topic_arousal_score,
        topic_valence_score
    ]
    to_insert.append(list_insert)
    
to_insert = np.array(to_insert)

Processing tweet number 1, time elapsed: 0:00:00.000898
Processing tweet number 2501, time elapsed: 0:16:36.925527
Processing tweet number 5001, time elapsed: 0:32:51.616548
Processing tweet number 7501, time elapsed: 0:48:44.361282
Processing tweet number 10001, time elapsed: 1:04:44.468805
Processing tweet number 12501, time elapsed: 1:20:45.447402
Processing tweet number 15001, time elapsed: 1:36:39.512961
Processing tweet number 17501, time elapsed: 1:52:15.557639
Processing tweet number 20001, time elapsed: 2:07:42.158853
Processing tweet number 22501, time elapsed: 2:23:03.006086
Processing tweet number 25001, time elapsed: 2:38:26.865141
Processing tweet number 27501, time elapsed: 2:53:49.249694
Processing tweet number 30001, time elapsed: 3:09:31.879624
Processing tweet number 32501, time elapsed: 3:25:47.216761
Processing tweet number 35001, time elapsed: 3:41:33.554764
Processing tweet number 37501, time elapsed: 3:57:13.887495
Processing tweet number 40001, time elapsed: 4:

ValueError: setting an array element with a sequence.

In [14]:
to_insert = np.array(to_insert, dtype=object)

arr_insert = np.array(to_insert)
arr_insert.shape

(55309, 6)

In [32]:
# DON'T DO THIS ONE -- I FORGOT TO ADD THE TOPICS 
for index in np.arange(arr_insert.shape[0]):
    df.iloc[index,col_map['arousal']] = arr_insert[index,0]
    df.iloc[index,col_map['valence']] = arr_insert[index,1]
    df.iloc[index,col_map['sentiment']] = arr_insert[index,2]
    df.iloc[index,col_map['topic_arousal']] = arr_insert[index,3]
    df.iloc[index,col_map['topic_valence']] = arr_insert[index,4]

df.head()

,text,topic,log_followers,log_retweets,engagement_flag,positive_emo,outrage_emo,net_emo_outrage,predicted_sentiment,negative_sentiment_prob,num_outrage_words_ext,num_outrage_words,arousal,valence,sentiment,topics,topic_arousal,topic_valence,outrage
user.description,,,,,,,,,,,,,,,,,,,
"Part of the #VastRightWingConspiracy -- thanks, @HillaryClinton. Better candidate: https://www.randpaul.com/ Retweeting not endorsement. Image: TV lover.","#BanTheBox means ""all is #forgiven""/NO MORE federal #NCIC checks when buying guns? Please consider it, Mr. President @POTUS. #forgiving",G,5.252273,0.0,0,0,0,0,4,0.265578,0,0,3.994752,4.674877,0.457199,"[(13, 0.131495800971), (27, 0.425939277591), (46, 0.0980153575283), (50, 0.266771786131), (13, 0.131394500046), (27, 0.425844825795), (46, 0.0980001633593), (50, 0.266982733022), (13, 0.1309867010...",1.144233,0.910757,0
War Veteran who loves his country. Never forget those who died in 'Benghazi'. 864th Engineer Battalion (Combat) (Heavy),BOYCOTT NOV 4 @ABC @CMAwards &gt; THREAT TO ALL GUN OWNERS - BOYCOTT HIS MUSIC &gt; @TheTimMcGraw GUN CONTROL EXTREMIST https://t.co/xMuE2o7vV7 …,G,9.936148,0.0,0,0,0,0,4,0.449931,0,0,5.726513,4.928830,0.486539,"[(6, 0.208727923843), (23, 0.0791057461022), (29, 0.0851475614148), (39, 0.391552329996), (50, 0.164953618131), (6, 0.208727689676), (23, 0.0791057173202), (29, 0.0851462087178), (39, 0.3915507425...",0.452195,0.388114,0
"Retired: cowboy,truck driver Served in the USAF, hobbies include Shooting sports and hunt and camp born in Calif.and sadly left in 1984","Savage: This is the Most Corrupt, Degenerate, Criminal Gov’t in American History https://t.co/qW12kzuBWH Take my guns lead first.",G,4.043051,0.0,0,0,0,0,0,0.583269,1,0,5.066761,4.056568,0.510775,"[(12, 0.13128764681), (27, 0.106913714881), (33, 0.325289099174), (50, 0.351661054287), (12, 0.131218108339), (27, 0.10687431201), (33, 0.325306845499), (50, 0.351752249303)]",0.561759,0.448320,0
Bartender...Bouncer...Autobody......Gravedigger.....Supermodal......OG.....Playboy.........:)\r\n\r\n\r\nDAM...NICE...GUY...MAFIA DEMOLITION INSTALATION,Guns N' Roses - November Rain https://t.co/w2pHuG0QBa via @YouTube,G,6.107023,0.0,0,0,0,0,4,0.470496,0,0,7.740000,3.660000,0.501845,"[(16, 0.190694556966), (29, 0.375986293763), (50, 0.274985815938), (16, 0.190722817926), (29, 0.376140201094), (50, 0.274803647647)]",0.366314,0.284348,0
Ming doesn't have a trajectory problem.,"I'll say it again: #Tarantino was merely correct. And, to his credit, he's sticking to his rhetorical guns. #BadCops https://t.co/7M4uTpXwyc",G,5.081404,0.0,0,0,0,0,4,0.362593,1,0,4.637737,4.979904,0.480766,"[(30, 0.223023678262), (49, 0.511015742518), (50, 0.186793912553), (30, 0.224087096145), (49, 0.511083915034), (50, 0.185662322154)]",0.508377,0.490181,0


In [52]:
df.iloc[18338,:]

text                       Liberals can't talk about evil, so they talk about guns @bayes46  https://t.co/wLCFQ4mslw
topic                                                                                                              G
log_followers                                                                                                5.80513
log_retweets                                                                                                       0
engagement_flag                                                                                                    1
positive_emo                                                                                                       0
outrage_emo                                                                                                        0
net_emo_outrage                                                                                                    0
predicted_sentiment                                             

In [53]:
df.to_csv('partially_measured.csv')

In [24]:
# DO THIS ONE OR SOME OTHER WAY YOU LIKE
for index in np.arange(arr_insert.shape[0]):
    df_sampled.iloc[index,col_map['arousal']] = arr_insert[index,0]
    df_sampled.iloc[index,col_map['valence']] = arr_insert[index,1]
    df_sampled.iloc[index,col_map['sentiment']] = arr_insert[index,2]
#    df_sampled.iloc[index,col_map['topics']] = arr_insert[index,3]
    df_sampled.iloc[index,col_map['topic_arousal']] = arr_insert[index,4]
    df_sampled.iloc[index,col_map['topic_valence']] = arr_insert[index,5]

df_sampled.head()

,user.description,text,topic,log_followers,log_retweets,engagement_flag,positive_emo,outrage_emo,net_emo_outrage,predicted_sentiment,negative_sentiment_prob,num_outrage_words_ext,num_outrage_words,arousal,valence,sentiment,topics,topic_arousal,topic_valence,outrage
3847,"#USMAlumna #DST cornbread & cabbage, a savage.",Why this bitch still making NEWS! Don't nobody give AF AB HER OPINIONS<f0><U+009F><U+0099><U+0084><f0><U+009F><U+0098><U+00A1>. Stop giving this bih headlines. https://t.co/xXvckQD2e8,G,6.957497,0.0,0,0,1,1,0,0.628052,1,0,4.243258,5.523956,0.520228,"[(6, 0.108965927292), (12, 0.0882638366989), (13, 0.0766967169164), (16, 0.119924533172), (20, 0.0514575371517), (26, 0.0517357081194), (30, 0.266809100622), (31, 0.049356041968), (48, 0.097998154...",0.418661,0.458308,0
3848,None,Find who did this........... https://t.co/91Uny7hRRJ,G,5.786897,0.0,0,0,0,0,0,0.506168,0,0,3.520000,6.450000,0.501028,"[(28, 0.754166666667)]",0.265133,0.309399,0


In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [33]:
df_sampled[df_sampled['sentiment']>.7]

,user.description,text,topic,log_followers,log_retweets,engagement_flag,positive_emo,outrage_emo,net_emo_outrage,predicted_sentiment,negative_sentiment_prob,num_outrage_words_ext,num_outrage_words,arousal,valence,sentiment,topics,topic_arousal,topic_valence,outrage
100213,"phd student; academic but prone to silliness. leftist. likes cats, food, books, music, tv, film. views own, RT=/=agree",Ugh https://t.co/ZHCdzq6pNC,G,7.273093,0.0,0,0,0,0,0,0.911123,0,0,NaN,NaN,0.705561,"[(32, 0.338888888887), (54, 0.338888888891), (32, 0.338888888887), (54, 0.338888888891)]",0.257134,0.291482,0


In [25]:
import pickle
outrage_scaler = pickle.load(open('../../scalers/outrage_scaler.pkl', 'rb'))
exp_outrage_scaler = pickle.load(open('../../scalers/ext_outrage_scaler.pkl', 'rb'))
valence_scaler = pickle.load(open('../../scalers/valence_scaler.pkl', 'rb'))
arousal_scaler = pickle.load(open('../../scalers/arousal_scaler.pkl', 'rb'))
emoji_scaler = pickle.load(open('../../scalers/emoji_scaler.pkl', 'rb'))
negative_sent_scaler = pickle.load(open('../../scalers/negative_sent_scaler.pkl', 'rb'))
topic_valence_scaler = pickle.load(open('../../scalers/topic_valence_scaler.pkl', 'rb'))
topic_arousal_scaler = pickle.load(open('../../scalers/topic_arousal_scaler.pkl', 'rb'))

In [26]:
df_sampled['arousal'].fillna(-5000, inplace=True)
df_sampled['valence'].fillna(-5000, inplace=True)
df_sampled['scaled_emoji']= df['net_emo_outrage'].apply(lambda x: 0 if x == -1 else (1 if x==1 else 0.5))
df_sampled['scaled_outrage'] = outrage_scaler.transform(df_sampled.num_outrage_words.values.reshape(-1,1))
df_sampled['scaled_ext_outrage'] = exp_outrage_scaler.transform(df_sampled.num_outrage_words_ext.values.reshape(-1,1))
df_sampled['scaled_valence'] = valence_scaler.transform(df_sampled.valence.values.reshape(-1,1))
df_sampled['scaled_arousal'] = arousal_scaler.transform(df_sampled.arousal.values.reshape(-1,1))
df_sampled['scaled_sentiment'] = negative_sent_scaler.transform(df_sampled.negative_sentiment_prob.values.reshape(-1,1))
df_sampled['scaled_topic_valence'] = topic_valence_scaler.transform(df_sampled.topic_valence.values.reshape(-1,1))
df_sampled['scaled_topic_arousal'] = topic_arousal_scaler.transform(df_sampled.topic_arousal.values.reshape(-1,1))

In [27]:
df_sampled['scaled_arousal']=df_sampled['scaled_arousal'].apply(lambda x: .5 if x <= -500 else x)
df_sampled['scaled_valence']=df_sampled['scaled_valence'].apply(lambda x: .5 if x <= -500 else x)

In [28]:
df_sampled['scaled_topic_valence']=1-df_sampled['scaled_topic_valence']
df_sampled['scaled_valence']=1-df_sampled['scaled_valence']

In [118]:
df_sampled.to_csv('sampled_scaled_outrage.csv')

In [33]:
df_sampled['outrage_index'] = .12*df_sampled['scaled_outrage'] + \
    .10*df_sampled['scaled_ext_outrage'] +\
    .14*df_sampled['scaled_valence'] +\
    .14*df_sampled['scaled_arousal'] +\
    .11*df_sampled['scaled_sentiment'] +\
    .10*df_sampled['scaled_emoji'] +\
    .15*df_sampled['scaled_topic_valence'] +\
    .14*df_sampled['scaled_topic_arousal']

In [34]:
df_sampled

,user.description,text,topic,log_followers,log_retweets,engagement_flag,positive_emo,outrage_emo,net_emo_outrage,predicted_sentiment,...,outrage,scaled_emoji,scaled_outrage,scaled_ext_outrage,scaled_valence,scaled_arousal,scaled_sentiment,scaled_topic_valence,scaled_topic_arousal,outrage_index
3847,"#USMAlumna #DST cornbread & cabbage, a savage.",Why this bitch still making NEWS! Don't nobody give AF AB HER OPINIONS<f0><U+009F><U+0099><U+0084><f0><U+009F><U+0098><U+00A1>. Stop giving this bih headlines. https://t.co/xXvckQD2e8,G,6.957497,0.0,0,0,1,1,0,...,0,1.0,0.5,0.6,0.348206,0.548224,0.628348,0.504188,0.365888,0.541471
3848,None,Find who did this........... https://t.co/91Uny7hRRJ,G,5.786897,0.0,0,0,0,0,0,...,0,0.5,0.5,0.5,0.238938,0.454780,0.505144,0.665283,0.231712,0.444919


In [31]:
df_sampled['scaled_outrage']= df_sampled['scaled_outrage'].apply(lambda x: .625 if x == 0.25 else (.75 if x==.5 else (.875 if x==.75 else (.5 if x==0 else 1))))
df_sampled['scaled_ext_outrage']= df_sampled['scaled_ext_outrage'].apply(lambda x: .6 if x == .2 else (.7 if x==.4 else (.8 if x==.6 else (.5 if x==0 else (.9 if x==.8 else 1)))))

In [ ]:
plt.style.use('ggplot')
plt.hist(df['negative_sentiment_prob'], bins=15, color='steelblue')
plt.title('Distribution of Negative Sentiment Probability')
plt.ylabel('Frequency')
plt.xlabel('Negative Sentiment Probability')
plt.savefig('tweet_sentiment.png',dpi=360)